# DATA ACQUISITION

In [ ]:
# Import required library
import pandas as pd
import numpy as np
import os
import yfinance as yf
from datetime import datetime

In this notebook we are going to acquire all data that we need for our project. We are going to consider a period of ten years
from 2014 to 2023. The reference index will be the NAS100; here we are going to acquire its price history and the history of its
shares over the last 10 years (where possible).

In [ ]:
url = 'https://en.wikipedia.org/wiki/Nasdaq-100'

# Read all tables from the webpage
tables = pd.read_html(url)

# Check all tables to find the right one
for i, table in enumerate(tables):
    print(f"Table {i}")
    print(table.head(), "\n")

Table 0
                                    0                                   1
0  Nasdaq-100 Index from 1985 to 2015  Nasdaq-100 Index from 1985 to 2015
1                          Foundation      January 31, 1985; 39 years ago
2                            Operator                        Nasdaq, Inc.
3                           Exchanges                              Nasdaq
4                      Trading symbol                                ^NDX 

Table 1
   Category  All-Time Highs[9]       All-Time Highs[9].1
0   Closing           20675.38  Wednesday, July 10, 2024
1  Intraday           20690.87  Wednesday, July 10, 2024 

Table 2
   Milestone[10] (closing) Date of Record (closing)[10]  Trading Days[10]
0                     1000                 July 8, 1997              3143
1                     2000             January 11, 1999               381
2                     3000            November 18, 1999               217
3                     4000             February 8, 2000      

In [ ]:
# Let's start by considering the shares that are currently componing our interest index
tickers = tables[4]["Ticker"]
tickers

0       ADBE
1        ADP
2       ABNB
3      GOOGL
4       GOOG
       ...  
96       WBA
97       WBD
98      WDAY
99       XEL
100       ZS
Name: Ticker, Length: 101, dtype: object

We are going to use data about added and removed security along the index history to get a list of all shares to be considered

In [ ]:
# Load data
nasdaq_constituent = pd.read_json("nasdaq_constituent.json")
nasdaq_constituent

,dateAdded,addedSecurity,removedTicker,removedSecurity,date,symbol,reason
0,"March 18, 2024",Linde plc,SPLK,Splunk Inc,2024-03-18,LIN,Annual Re-ranking
1,"December 18, 2023",,SGEN,Seagen Inc,2023-12-18,TTWO,Annual Re-ranking
2,"December 18, 2023",CDW Corporation,ALGN,Align Technology Inc,2023-12-18,CDW,Annual Re-ranking
3,"December 18, 2023",,EBAY,eBay Inc,2023-12-18,CCEP,Annual Re-ranking
4,"December 18, 2023",DoorDash Inc,ENPH,Enphase Energy Inc,2023-12-18,DASH,Annual Re-ranking
...,...,...,...,...,...,...,...
401,"March 6, 1995",HBO & Co,SON,Sonoco Products Co,1995-03-06,HBOC,Annual Re-ranking
402,"February 27, 1995",,SMLS,"SciMed Life Systems, Inc",1995-02-27,KLAC,Annual Re-ranking
403,"February 13, 1995",A Schulman Inc,QVCN,"QVC, Inc.",1995-02-13,SHLM,Annual Re-ranking
404,"January 27, 1995",RPM Int'l Inc,DG,Dollar General Corp,1995-01-27,RPM,Annual Re-ranking


In [ ]:
tickers_added = nasdaq_constituent["symbol"].unique().tolist()
tickers_removed = nasdaq_constituent["removedTicker"].unique().tolist()

In [ ]:
S1 = set(tickers)
S2 = set(tickers_added)
S3 = set(tickers_removed)
S4 = S2.union(S3)
new_tickers = list(S4.difference(S1))

In [ ]:
len(new_tickers)

322

In [ ]:
tickers_list = tickers.to_list()
total_tickers = tickers_list + new_tickers

In [ ]:
total_tickers.remove('')

In [ ]:
len(total_tickers)

422

In [ ]:
# Define the end date
end_date = datetime(2023, 12, 31)
# Define the start date (10 years before the end date)
start_date = datetime(2013, 12, 31)

# Dictionary to store the valid tickers with sufficient historical data
valid_tickers = {}

for ticker in total_tickers:
    # Download data
    df = yf.download(ticker, start = start_date.strftime('%Y-%m-%d'), end = end_date.strftime('%Y-%m-%d'), progress = True)

    df = df[['Close']]

    valid_tickers[ticker] = df

# Combine valid tickers into a single DataFrame
if valid_tickers:
    combined_data = pd.concat(valid_tickers, axis=1)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

$CEXP: possibly delisted; No price data found  (1d 2013-12-31 -> 2023-12-31)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['DISH']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['GLBC']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2013-12-31 -> 2023-12-31)')


$GLBC: possibly delisted; No price data found  (1d 2013-12-31 -> 2023-12-31)


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['CERN']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['SXCL']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2013-12-31 -> 2023-12-31)')


...

[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['SPLK']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['FLIR']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['CSCC']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2013-12-31 -> 2023-12-31)')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['COMR']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2013-12-31 -> 2023-12-31)')


$CSCC: possibly delisted; No price data found  (1d 2013-12-31 -> 2023-12-31)
$COMR: possibly delisted; No price data found  (1d 2013-12-31 -> 2023-12-31)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['BMET']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2013-12-31 -> 2023-12-31)')


$BMET: possibly delisted; No price data found  (1d 2013-12-31 -> 2023-12-31)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['TECD']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['EDCI']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['PDLI']: YFTzMissingErro

In [ ]:
combined_data

,ADBE,ADP,ABNB,GOOGL,GOOG,AMZN,AMD,AEP,AMGN,ADI,...,ADTN,PALM,VTRS,LOGI,DIGI,EDCI,PDLI,TWX,NTAP,ADCT
Date,,,,,,,,,,,,,,,,,,,,,
2013-12-31 00:00:00,59.880001,70.939423,NaN,28.045795,NaN,19.939501,3.870000,46.740002,114.080002,50.930000,...,27.010000,NaN,43.400002,13.690000,0.0014,NaN,NaN,66.841148,41.139999,NaN
2014-01-02 00:00:00,59.290001,70.114136,NaN,27.855856,NaN,19.898500,3.950000,46.130001,115.800003,49.279999,...,26.420000,NaN,42.580002,13.350000,0.0030,NaN,NaN,65.892029,40.419998,NaN
2014-01-03 00:00:00,59.160000,70.877960,NaN,27.652653,NaN,19.822001,4.000000,46.110001,114.470001,49.610001,...,26.510000,NaN,42.720001,13.400000,0.0016,NaN,NaN,65.834511,40.330002,NaN
2014-01-06 00:00:00,58.119999,70.070236,NaN,27.960960,NaN,19.681499,4.130000,46.080002,113.480003,49.330002,...,26.549999,NaN,42.259998,13.590000,0.0016,NaN,NaN,65.345566,40.150002,NaN
2014-01-07 00:00:00,58.970001,70.921860,NaN,28.500000,NaN,19.901501,4.180000,46.380001,116.430000,49.590000,...,26.780001,NaN,42.980000,14.050000,0.0016,NaN,NaN,64.712814,40.820000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22 00:00:00,598.750000,230.970001,140.800003,141.490005,142.720001,153.419998,139.600006,80.379997,284.160004,197.100006,...,7.150000,NaN,10.650000,92.580002,0.0001,NaN,NaN,NaN,89.400002,1.48
2023-12-26 00:00:00,598.260010,232.539993,138.720001,141.520004,142.820007,153.410004,143.410004,80.730003,283.899994,198.869995,...,7.330000,NaN,10.710000,93.230003,0.0001,NaN,NaN,NaN,89.379997,1.65
2023-12-27 00:00:00,596.080017,231.610001,136.550003,140.369995,141.440002,153.339996,146.070007,80.809998,286.529999,199.350006,...,7.300000,NaN,10.770000,94.559998,0.0001,NaN,NaN,NaN,88.599998,1.72


using a specific library we retrieve the history of trading days for the considered market and look if we have the same number of days for the downloaded data:

In [ ]:
import pandas_market_calendars as mcal

nas = mcal.get_calendar("NASDAQ")
trading_days = nas.valid_days(start_date = start_date, end_date = end_date)
trading_days

DatetimeIndex(['2013-12-31 00:00:00+00:00', '2014-01-02 00:00:00+00:00',
               '2014-01-03 00:00:00+00:00', '2014-01-06 00:00:00+00:00',
               '2014-01-07 00:00:00+00:00', '2014-01-08 00:00:00+00:00',
               '2014-01-09 00:00:00+00:00', '2014-01-10 00:00:00+00:00',
               '2014-01-13 00:00:00+00:00', '2014-01-14 00:00:00+00:00',
               ...
               '2023-12-15 00:00:00+00:00', '2023-12-18 00:00:00+00:00',
               '2023-12-19 00:00:00+00:00', '2023-12-20 00:00:00+00:00',
               '2023-12-21 00:00:00+00:00', '2023-12-22 00:00:00+00:00',
               '2023-12-26 00:00:00+00:00', '2023-12-27 00:00:00+00:00',
               '2023-12-28 00:00:00+00:00', '2023-12-29 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=2517, freq=None)

In [ ]:
combined_data.columns = combined_data.columns.get_level_values(0)

Probably not all the downloaded data are going to be useable because of missing values.
Let's define a function to check if data for a stock respect that there are non-NaN after the first historical data present in the time series and if there is enough data to perform a rolling regression:

In [ ]:
def is_valid_column(series):
    """checks whether a column is valid according to the specified conditions:"""
    # Verify if there are NaN values
    if series.isnull().any():
        # Find first index that is not Nan
        first_non_nan_index = series.first_valid_index()
        if first_non_nan_index is not None:
            # Check whether all values after the first non-NaN are also non-NaN
            return series[first_non_nan_index:].isnull().sum() == 0
        else:
            # If all values are NaN, consider the column as invalid
            return False
    return True

# Filter df columns
valid_columns = [col for col in combined_data.columns if is_valid_column(combined_data[col])]
df_filtered = combined_data[valid_columns]
df_filtered

,ADBE,ADP,ABNB,GOOGL,GOOG,AMZN,AMD,AEP,AMGN,ADI,...,ASND,MCIC,MAT,KELYA,ADTN,VTRS,LOGI,DIGI,NTAP,ADCT
Date,,,,,,,,,,,,,,,,,,,,,
2013-12-31 00:00:00,59.880001,70.939423,NaN,28.045795,NaN,19.939501,3.870000,46.740002,114.080002,50.930000,...,NaN,0.0024,47.580002,24.940001,27.010000,43.400002,13.690000,0.0014,41.139999,NaN
2014-01-02 00:00:00,59.290001,70.114136,NaN,27.855856,NaN,19.898500,3.950000,46.130001,115.800003,49.279999,...,NaN,0.0030,47.389999,24.610001,26.420000,42.580002,13.350000,0.0030,40.419998,NaN
2014-01-03 00:00:00,59.160000,70.877960,NaN,27.652653,NaN,19.822001,4.000000,46.110001,114.470001,49.610001,...,NaN,0.0029,46.869999,24.680000,26.510000,42.720001,13.400000,0.0016,40.330002,NaN
2014-01-06 00:00:00,58.119999,70.070236,NaN,27.960960,NaN,19.681499,4.130000,46.080002,113.480003,49.330002,...,NaN,0.0029,46.619999,24.350000,26.549999,42.259998,13.590000,0.0016,40.150002,NaN
2014-01-07 00:00:00,58.970001,70.921860,NaN,28.500000,NaN,19.901501,4.180000,46.380001,116.430000,49.590000,...,NaN,0.0033,46.040001,24.660000,26.780001,42.980000,14.050000,0.0016,40.820000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22 00:00:00,598.750000,230.970001,140.800003,141.490005,142.720001,153.419998,139.600006,80.379997,284.160004,197.100006,...,124.550003,0.0006,19.120001,21.660000,7.150000,10.650000,92.580002,0.0001,89.400002,1.48
2023-12-26 00:00:00,598.260010,232.539993,138.720001,141.520004,142.820007,153.410004,143.410004,80.730003,283.899994,198.869995,...,127.120003,0.0005,19.100000,21.900000,7.330000,10.710000,93.230003,0.0001,89.379997,1.65
2023-12-27 00:00:00,596.080017,231.610001,136.550003,140.369995,141.440002,153.339996,146.070007,80.809998,286.529999,199.350006,...,127.099998,0.0005,18.990000,21.990000,7.300000,10.770000,94.559998,0.0001,88.599998,1.72


Let's compute logarithmic returns for all selected stocks:

In [ ]:
log_returns = np.log( df_filtered / df_filtered.shift(1))
log_returns

,ADBE,ADP,ABNB,GOOGL,GOOG,AMZN,AMD,AEP,AMGN,ADI,...,ASND,MCIC,MAT,KELYA,ADTN,VTRS,LOGI,DIGI,NTAP,ADCT
Date,,,,,,,,,,,,,,,,,,,,,
2013-12-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-02 00:00:00,-0.009902,-0.011702,NaN,-0.006796,NaN,-0.002058,0.020461,-0.013137,0.014965,-0.032934,...,NaN,0.223144,-0.004001,-0.013320,-0.022086,-0.019075,-0.025149,0.762140,-0.017656,NaN
2014-01-03 00:00:00,-0.002195,0.010835,NaN,-0.007322,NaN,-0.003852,0.012579,-0.000434,-0.011552,0.006674,...,NaN,-0.033902,-0.011033,0.002840,0.003401,0.003283,0.003738,-0.628609,-0.002229,NaN
2014-01-06 00:00:00,-0.017736,-0.011461,NaN,0.011088,NaN,-0.007113,0.031983,-0.000651,-0.008686,-0.005660,...,NaN,0.000000,-0.005348,-0.013461,0.001508,-0.010826,0.014080,0.000000,-0.004473,NaN
2014-01-07 00:00:00,0.014519,0.012081,NaN,0.019095,NaN,0.011116,0.012034,0.006489,0.025664,0.005257,...,NaN,0.129212,-0.012519,0.012651,0.008626,0.016894,0.033288,0.000000,0.016550,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22 00:00:00,-0.002319,0.008609,-0.009120,0.007591,0.006467,-0.002734,-0.002218,0.004239,0.017144,0.004169,...,0.022327,0.000000,0.003668,0.007414,0.011252,0.000000,-0.006996,0.000000,-0.001676,0.145182
2023-12-26 00:00:00,-0.000819,0.006774,-0.014883,0.000212,0.000700,-0.000065,0.026926,0.004345,-0.000915,0.008940,...,0.020424,-0.182322,-0.001047,0.011019,0.024863,0.005618,0.006996,0.000000,-0.000224,0.108733
2023-12-27 00:00:00,-0.003651,-0.004007,-0.015767,-0.008159,-0.009710,-0.000456,0.018378,0.000990,0.009221,0.002411,...,-0.000157,0.000000,-0.005776,0.004101,-0.004101,0.005587,0.014165,0.000000,-0.008765,0.041549


Finally, we download the data of the index chosen as reference and calculate the logarithmic returns:

In [ ]:
nas100 = yf.download('^NDX', start = start_date, end = end_date)
nas100

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-12-31,3576.239990,3592.000000,3575.250000,3592.000000,3592.000000,1401140000
2014-01-02,3575.600098,3577.030029,3553.649902,3563.570068,3563.570068,1738820000
2014-01-03,3564.939941,3567.510010,3537.610107,3538.729980,3538.729980,1667480000
2014-01-06,3539.020020,3542.520020,3512.449951,3526.959961,3526.959961,2292840000
2014-01-07,3539.290039,3562.989990,3535.500000,3557.850098,3557.850098,2278220000
...,...,...,...,...,...,...
2023-12-22,16799.019531,16839.250000,16703.570312,16777.400391,16777.400391,4796600000
2023-12-26,16816.779297,16907.509766,16813.570312,16878.460938,16878.460938,6120600000
2023-12-27,16896.140625,16922.009766,16859.550781,16906.800781,16906.800781,7480170000


In [ ]:
# Compute log returns
nas100['log_returns'] = np.log(nas100.Close / nas100.Close.shift(1))
nas100 = nas100[["Close", "log_returns"]]
nas100

,Close,log_returns
Date,,
2013-12-31,3592.000000,NaN
2014-01-02,3563.570068,-0.007946
2014-01-03,3538.729980,-0.006995
2014-01-06,3526.959961,-0.003332
2014-01-07,3557.850098,0.008720
...,...,...
2023-12-22,16777.400391,0.001192
2023-12-26,16878.460938,0.006006
2023-12-27,16906.800781,0.001678


Save all acquired data in a csv format:

In [ ]:
df_filtered.to_csv("Data/StocksPrices.csv")
log_returns.to_csv("Data/StocksReturns.csv")
nas100.to_csv("Data/nas100.csv")